In [1]:
%pylab inline
import homer
import pandas as pd
import numpy as np
import dask.bag as db
from dask.diagnostics import ProgressBar
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
import json
from dask.diagnostics import visualize
import dask
import pickle
from itertools import combinations
import re
import dateutil.parser
from nltk.tokenize import RegexpTokenizer
import ujson
import dask.dataframe as dd
import itertools
from dask import delayed
from bokeh.io import output_notebook
import collections
import glob
output_notebook()

Populating the interactive namespace from numpy and matplotlib


Loading BokehJS ...

In [10]:
dates = [20150615]
thresholds = [1]
tw_files='tests/resources/small_tw_file*.gz'


In [15]:
dates = [20150615]
thresholds = [10]
tw_files=['/Users/houghton/Desktop/tw/posts_sample_20150615_162235_aa.gz',
          '/Users/houghton/Desktop/tw/posts_sample_20150615_162235_aa.gz']


In [21]:
#dates = [20150615]
thresholds = [50]
tw_files='/Users/houghton/Desktop/tw/posts_sample_201506*.gz'         


In [ ]:
dates = [20150615]
thresholds = [10]
tw_files='/Users/houghton/Desktop/tw/posts_sample_20150615_162235_a*.gz'      

In [3]:
dates = [20150615, 20150616, 20150617, 20150618, 20150619, 20150620, 
         20150621, 20150622, 20150623, 20150624, 20150625]

sw_file = "homer/stopwords.pickle"
tw_stopwords = pickle.load(open(sw_file, "rb"))

tokenizer = RegexpTokenizer(r'\w+')
matcher = re.compile(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*')

languages = ['en']
def fold_pairs(counter, json_string):
    parsed_json = ujson.loads(json_string)
    if ('lang' in parsed_json and
            'text' in parsed_json and
            parsed_json['lang'] in languages):
        text = parsed_json['text'].lower()

        # remove hyperlinks
        text = matcher.sub('', text)

        # tokenize, dropping punctuation
        tokens = tokenizer.tokenize(text)  # this should release GIL - be ok for parallelization

        # drop stopwords
        tokens = filter(lambda x: x not in tw_stopwords, tokens)

        date = int(dateutil.parser.parse(parsed_json['created_at']).strftime("%Y%m%d"))
        
        if date not in counter:
            counter[date] = collections.Counter()
        for pair in combinations(tokens, 2):
            counter[date][tuple(sorted(pair))] += 1

    return counter

def merge_folds(a, b):
    if isinstance(a, dict):
        series = {key:pd.Series(c) for key, c in a.items() if len(c)}
        a = pd.concat(series.values(), keys=series.keys())
        
    if isinstance(b, dict):
        series = {key:pd.Series(c) for key, c in b.items() if len(c)}
        b = pd.concat(series.values(), keys=series.keys())
        
    return a.add(b, fill_value=0)

In [22]:
lines = db.read_text(tw_files, compression='gzip', collection=True)

initial_counter = {date:collections.Counter() for date in dates}
counters = lines.fold(binop=fold_pairs, combine=merge_folds, initial=initial_counter)

In [23]:
with ProgressBar():
    counts = counters.compute()
    counts_df = dd.from_pandas(counts.reset_index(level=[1,2]), npartitions=10)
    counts_df.columns = ['W1', 'W2', 'Count']
    counts_df = counts_df[counts_df['Count'] > thresholds[0]]

    def name(i):
        return str(counts_df.divisions[i]) + '_' + str(counts_df.divisions[i+1])

    counts_df.to_csv('/Users/houghton/Desktop/tw/charleston/wel_*.csv.gz', 
                     name_function = name,
                     compression='gzip')

[####################################### ] | 99% Completed |  2hr 27min 43.3s


Process ForkPoolWorker-43:
KeyboardInterrupt
Process ForkPoolWorker-47:
Process ForkPoolWorker-42:
Process ForkPoolWorker-44:
Process ForkPoolWorker-45:
Process ForkPoolWorker-46:
Process ForkPoolWorker-41:
Process ForkPoolWorker-48:
Traceback (most recent call last):
  File "/Users/houghton/anaconda/envs/py3k/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/houghton/anaconda/envs/py3k/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/Users/houghton/anaconda/envs/py3k/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Users/houghton/anaconda/envs/py3k/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
Traceback (most recent call last):
  File "/Users/houghton/anaconda/envs/py3k/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter

KeyboardInterrupt: 

In [ ]:
with Profiler() as prof, ResourceProfiler(dt=5) as rprof, CacheProfiler() as cprof:
    counts = counters.compute()
visualize([prof, rprof, cprof])  

In [ ]:
with Profiler() as prof, ResourceProfiler(dt=5) as rprof, CacheProfiler() as cprof:
    counts = counters.compute()
visualize([prof, rprof, cprof])

In [ ]:
counts_df = dd.from_pandas(counts.reset_index(level=[1,2]), npartitions=10)
#counts_df.to_hdf('/Users/houghton/Desktop/tw/charleston/test.hdf', '/clusters', dropna=True)

In [ ]:
counts_df.columns = ['W1', 'W2', 'Count']

In [ ]:
counts_df.to_csv()

In [ ]:
with ProgressBar():
    counts = counters.compute()

In [ ]:
threshold=50
def find_clusters(date, counter, threshold):
    series = pd.Series(counter)
    over_thresh = series[series>threshold]
    over_thresh.to_hdf('/Users/houghton/Desktop/tw/charleston/test_%i_%i.hdf' %(date, threshold), '/clusters', dropna=True)

    #clusters = homer.clusterer.find_clusters(pd.DataFrame(pairs))
    #clusters.to_hdf('/Users/houghton/Desktop/tw/charleston/test_%i_%i.hdf' %(date, threshold), '/clusters', dropna=True)
    
lazy_find_clusters = delayed(find_clusters)

In [ ]:
with ProgressBar():
    collector = []
    for date, counter in counters.compute().items():
        collector.append(lazy_find_clusters(date, counter, threshold))
    dask.compute(*collector)

In [ ]:
#tw_files='tests/resources/testfile.txt'
# tw_files=['/Users/houghton/Desktop/tw/posts_sample_20150615_162235_aa.gz',
#            '/Users/houghton/Desktop/tw/posts_sample_20150615_162235_aa.gz']

# tw_files='/Users/houghton/Desktop/tw/posts_sample_20150615_162235_a*.gz'

tw_files='/Users/houghton/Desktop/tw/posts_sample_201506*.gz'
languages=['en']

save_threshold = 10

sw_file = "homer/stopwords.pickle"
tw_stopwords = pickle.load(open(sw_file, "rb"))

In [ ]:
df = homer.parser.get_weighted_edgelist(tw_files, languages=['en'], save_threshold=10)

In [ ]:
groups = df.groupby('Date')

In [ ]:

clusters = groups.apply(homer.clusterer.find_clusters_for_any_threshold)

In [ ]:
cluster = groups.apply(lambda x: homer.find_clusters_over_threshold(x, 10))

In [ ]:
# frequencies on daily, parse json separate
dates = ['20160618']
thresholds = [10, 15]

tokenizer = RegexpTokenizer(r'\w+')
def find_pairs(parsed_json):
    text = parsed_json['text'].lower()

    # remove hyperlinks
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)

    # tokenize, dropping punctuation
    tokens = tokenizer.tokenize(text)  # this should release GIL - be ok for parallelization

    # drop stopwords
    tokens = filter(lambda x: x not in tw_stopwords, tokens)

    date = dateutil.parser.parse(parsed_json['created_at']).strftime("%Y%m%d")

    sets = [tuple([date] + sorted(pair)) for pair in combinations(tokens, 2)]
    return sets

messages = db.read_text(tw_files, compression='gzip').map(ujson.loads)

def selector(msg):
    return 'lang' in msg and 'text' in msg and msg['lang'] in languages

selection = messages.filter(selector)
sets = selection.map(find_pairs).concat()

for date in dates:
    frequencies_on_date = delayed(sets.filter(lambda x: x[0] == date).frequencies())
    #for threshold in thresholds:
    #    above_threshold = frequencies_on_date.filter(lambda x: x[1]>threshold)
    #    pairs = above_threshold.pluck([0][1], [0][2])
        
    break
    

In [ ]:
# frequencies on daily, parse json integral
dates = [20150615]
thresholds = [10, 15]
languages = ['en']

tokenizer = RegexpTokenizer(r'\w+')
matcher = re.compile(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*')

def find_pairs(json_string):
    msg = ujson.loads(json_string)
    if 'lang' in msg and 'text' in msg and msg['lang'] in languages:
    
        text = msg['text'].lower()

        # remove hyperlinks
        text = matcher.sub('', text)

        # tokenize, dropping punctuation
        tokens = tokenizer.tokenize(text)  # this should release GIL - be ok for parallelization

        # drop stopwords
        tokens = filter(lambda x: x not in tw_stopwords, tokens)

        date = int(dateutil.parser.parse(msg['created_at']).strftime("%Y%m%d"))

        sets = [tuple([date] + sorted(pair)) for pair in combinations(tokens, 2)]
        return sets
    
    else:
        return []


sets = db.read_text(tw_files, compression='gzip').map(find_pairs).concat()


for date in dates:
    sets_on_date = sets.filter(lambda x: x[0] == date)
    frequencies_on_date = sets_on_date.frequencies()
    for threshold in thresholds:
        above_threshold = frequencies_on_date.filter(lambda x: x[1]>threshold)
        pairs = above_threshold.pluck([0]).pluck([1, 2])
        break
    break

#frequencies_on_date.take(3)
    

In [ ]:
# frequencies on whole set
dates = [20150615]
thresholds = [10, 15]
languages = ['en']

tokenizer = RegexpTokenizer(r'\w+')
def find_pairs(json_string):
    msg = ujson.loads(json_string)
    if 'lang' in msg and 'text' in msg and msg['lang'] in languages:
    
        text = msg['text'].lower()

        # remove hyperlinks
        text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)

        # tokenize, dropping punctuation
        tokens = tokenizer.tokenize(text)  # this should release GIL - be ok for parallelization

        # drop stopwords
        tokens = filter(lambda x: x not in tw_stopwords, tokens)

        date = int(dateutil.parser.parse(msg['created_at']).strftime("%Y%m%d"))

        sets = [tuple([date] + sorted(pair)) for pair in combinations(tokens, 2)]
        return sets
    
    else:
        return []


    

sets = db.read_text(tw_files, compression='gzip').map(find_pairs).concat()
frequencies = sets.frequencies()

template = pd.DataFrame([{'Date': 20170121, 'W1': 'Toad', 'W2': u'Bug', 'Count': 21}],
                                columns=['Date', 'W1', 'W2', 'Count'])

collector = []
for date, threshold in itertools.product(dates, thresholds):
    selection = frequencies.filter(lambda x: x[0][0] == date and x[1]>threshold)
    expand = frequencies.map(lambda x: (x[0][0], x[0][1], x[0][2], x[1]))
    uw_df = expand.to_dataframe(template)
    collector.append(delayed(homer.clusterer.find_clusters(uw_df[['W1', 'W2']]).assign(Threshold=threshold)))

clusters = dd.from_delayed(collector, meta=template)
clusters.to_hdf('/Users/houghton/Desktop/tw/charleston/test.hdf', '/clusters', dropna=True)

visualize([prof, rprof, cprof])     

In [ ]:
# via dataframe
#data goes into the 25th of the month...

dates = [20150616, 20150617, 20150618, 20150619, 20150620, 20150621, 20150622]
thresholds = [120]

tokenizer = RegexpTokenizer(r'\w+')
def find_pairs(json_string):
    msg = ujson.loads(json_string)
    if 'lang' in msg and 'text' in msg and msg['lang'] in languages:
    
        text = msg['text'].lower()

        # remove hyperlinks
        text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)

        # tokenize, dropping punctuation
        tokens = tokenizer.tokenize(text)  # this should release GIL - be ok for parallelization

        # drop stopwords
        tokens = filter(lambda x: x not in tw_stopwords, tokens)

        date = int(dateutil.parser.parse(msg['created_at']).strftime("%Y%m%d"))

        sets = [tuple([date] + sorted(pair)) for pair in combinations(tokens, 2)]
        return sets
    
    else:
        return []



sets = db.read_text(tw_files, compression='gzip').map(find_pairs).concat()
frequencies = sets.frequencies()
expand = frequencies.map(lambda x: (x[0][0], x[0][1], x[0][2], x[1]))

template = pd.DataFrame([{'Date': 20170121, 'W1': 'Toad', 'W2': u'Bug', 'Count': 21}],
                            columns=['Date', 'W1', 'W2', 'Count'])

weighted_df = expand.to_dataframe(template)

lazy_find_clusters = delayed(homer.clusterer.find_clusters)

collector = []
for date, threshold in itertools.product(dates, thresholds):
    on_date = weighted_df[weighted_df['Date']==date]
    unweighted_df = on_date[on_date['Count'] >= threshold][['W1', 'W2']]
    collector.append(lazy_find_clusters(unweighted_df).assign(Threshold=threshold))


template2 = pd.DataFrame(data=[{'Set':'Ho Bean', 'k':5, 'threshold': 10, 'Date':20170101}],
                         columns=['Set', 'k', 'threshold', 'Date'])    

clusters = dd.from_delayed(collector, meta=template2)
#clusters.to_hdf('/Users/houghton/Desktop/tw/charleston/test.hdf', '/clusters', dropna=True)
    
#visualize([prof, rprof, cprof])     

In [ ]:
with ProgressBar():
    clusters.to_hdf('/Users/houghton/Desktop/tw/charleston/full_set_t100.hdf', '/clusters', dropna=True)

In [ ]:
with Profiler() as prof, ResourceProfiler(dt=5) as rparof, CacheProfiler() as cprof:
    clusters.to_hdf('/Users/houghton/Desktop/tw/charleston/full_set.hdf', '/clusters', dropna=True)
visualize([prof, rprof, cprof])     

In [ ]:
clusters.npartitions

In [ ]:
clusters.visualize()

In [ ]:
dask.compute

In [ ]:
unweighted_df.visualize()

In [ ]:
clusters.head()

In [ ]:
unweighted_df.visualize()

In [ ]:
thresholds = weighted_edge_list['Count'].unique()
clusters_collector = []
for t in np.array(thresholds):
    if t < min_threshold:
        continue
    cluster = delayed(find_clusters_over_threshold)(weighted_edge_list, t)
    clusters_collector.append(cluster)

clusters = dd.from_delayed(clusters_collector)
return clusters

In [ ]:
template = pd.DataFrame(data=[{'Set':'Ho Bean', 'k':5, 'threshold': 10, 'Date':20170101}],
                        columns=['Set', 'k', 'threshold', 'Date'])

collector = []
for date in [20160618]:
    selection = w_el[w_el.Date == date]
    df = homer.clusterer.find_clusters_for_any_threshold(selection,
                                         min_threshold=10)

    df_2 = df.assign(Date=date)
    collector.append(df_2)

clusters = dd.from_delayed(collector, meta=template)        
#clusters = dd.concat(collector)        
#clusters = dd.concat(collector, interleave_partitions=True)
#clusters = clusters.repartition(npartitions=1)  # Todo: This partitioning is problematic
#clusters = clusters.reset_index(drop=True) #Todo: need to set an index which is unlikely to
# also be a word in a cluster.
#clusters.to_hdf(output_globstring, '/clusters', dropna=True)

In [ ]:
with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof:
    groups.Count.mean().compute()
visualize([prof, rprof, cprof]) 

In [ ]:
clusters.visualize()

In [ ]:
with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof:
    clusters.compute()

In [ ]:
with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof:
    df.to_hdf('/Users/houghton/Desktop/tw/charleston/wel.hdf', key='/weighted_edge_list', dropna=True)

In [ ]:
df.head()

In [ ]:
df.categorize('Date')

In [ ]:
df.repartition(divisions=[0, 100])

In [ ]:
with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof:
    df.compute()

visualize([prof, rprof, cprof])  

In [ ]:
visualize([prof, rprof, cprof]) 

In [ ]:
df = homer.get_weighted_edgelist(tw_files, languages=['en'], save_threshold=10)
df.visualize()

In [ ]:
#with ProgressBar():
with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof:
    homer.build_weighted_edgelist_db(tw_files, 
                                     output_globstring='/Users/houghton/Desktop/tw/charleston/weighted_edgelists_*.hdf',
                                     save_threshold=10)

In [ ]:
template = pd.DataFrame([{'Date': 20170121, 'W1': 'Toad', 'W2': u'Bug', 'Count': 21}],
                            columns=['Date', 'W1', 'W2', 'Count'])
save_threshold = 10

In [ ]:


cooccurences = db.read_text(tw_files).map(homer.parser.get_message_cooccurences,
                                          languages=['en']).concat()
frequencies = cooccurences.frequencies()
over_threshold = frequencies.filter(lambda x: x[1] >= save_threshold)
expand = over_threshold.map(lambda x: (x[0][0], x[0][1], x[0][2], x[1]))
df = expand.to_dataframe(template)

In [ ]:
with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof:
    df.compute()
    
visualize([prof, rprof, cprof])    

In [ ]:
from dask.diagnostics import visualize
visualize([prof, rprof, cprof])

In [ ]:
with ProgressBar():
    df.head()

In [ ]:
template = pd.DataFrame([{'Date':20170121, 'W1':'Toad', 'W2':u'Bug', 'Count':21}], 
                        columns=['Date', 'W1', 'W2', 'Count'])

In [ ]:
template

In [ ]:
cooccurences = db.read_text(tw_files).map(homer.parser.get_message_cooccurences,
                                          languages=languages).concat()

#cooccurences.take(3)
frequencies = cooccurences.frequencies()

over_threshold = frequencies.filter(lambda x: x[1] >= keep_threshold)
expand = over_threshold.map(lambda x: (x[0][0], x[0][1], x[0][2], x[1]))
df = expand.to_dataframe(template)

#frequencies = cooccurences.frequencies()
# dicts = frequencies.map(
#        lambda x: {'Date': int(x[0][0]),
#                    'W1': str(x[0][1]),
#                    'W2': str(x[0][2]),
#                    'Count': int(x[1])})
#     df = dicts.to_dataframe()


with ProgressBar():
#with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof:
    #out = over_threshold.take(3)
    #size = over_threshold.count().compute()
    df.head()
    
#print(size)

#out

In [ ]:
cooccurences = db.read_text(tw_files).map(homer.parser.get_message_cooccurences,
                                          languages=languages).concat()

#cooccurences.take(3)
frequencies = cooccurences.frequencies()

over_threshold = frequencies.filter(lambda x: x[1] >= keep_threshold)
expand = over_threshold.map(lambda x: (*x[0], x[1]))
df = expand.to_dataframe(template)

#frequencies = cooccurences.frequencies()
# dicts = frequencies.map(
#        lambda x: {'Date': int(x[0][0]),
#                    'W1': str(x[0][1]),
#                    'W2': str(x[0][2]),
#                    'Count': int(x[1])})
#     df = dicts.to_dataframe()


with ProgressBar():
#with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof:
    #out = over_threshold.take(3)
    #size = over_threshold.count().compute()
    df.head()
    
#print(size)

#out

In [ ]:
df.visualize()

In [ ]:
df = over_threshold.to_dataframe(columns=[['Date', 'W1', 'W2'], 'Count'])

In [ ]:
df.head()

In [ ]:
from dask.diagnostics import visualize
visualize([prof, rprof, cprof])

In [ ]:
over_threshold.visualize()

In [ ]:
cooccurences = db.read_text(tw_files).map(homer.parser.get_message_cooccurences,
                                          languages=languages).concat()

#cooccurences.take(3)
frequencies = cooccurences.frequencies()

#over_threshold = frequencies.filter(lambda x: x[1] > keep_threshold)
frequencies.remove(lambda x: x[1] < keep_threshold)

#frequencies = cooccurences.frequencies()
# dicts = frequencies.map(
#        lambda x: {'Date': int(x[0][0]),
#                    'W1': str(x[0][1]),
#                    'W2': str(x[0][2]),
#                    'Count': int(x[1])})
#     df = dicts.to_dataframe()

with ProgressBar():
    out = frequencies.take(3)
    #size = frequencies.count().compute()
    
print(size)

out

In [ ]:
over_threshold.visualize()

In [ ]:
df = homer.parser.get_weighted_edgelist(tw_files='tests/resources/testfile.txt',
                                        languages=['en'])


In [ ]:
collection = homer.Homer(weighted_edge_list_globstring='tests/resources/Making_Connections_Generated_Data.hdf')                   

In [ ]:
collection.compute_clusters('working/MC_gen_clusters_*.hdf', min_threshold=2)

In [ ]:
collection.compute_relations('working/MC_gen_relations_*.hdf')

In [ ]:
collection.compute_transition_clusters('working/MC_gen_transitions_*.hdf', min_threshold=3)

In [ ]:
collection.transition_clusters.visualize()

In [ ]:
collection = homer.Homer(clusters_globstring='working/MC_gen_clusters_*.hdf', 
                         transition_clusters_globstring='working/MC_gen_transitions_*.hdf')

In [ ]:
collection.compute_transition_list('working/MC_gen_transitions_list.pickle')

In [ ]:
collection.compute_tree(tree_filename='working/MC_gen_tree.json.gz')

In [ ]:
%debug


In [ ]:
collection = homer.Homer(weighted_edge_list_globstring='tests/resources/Making_Connections_Generated_Data.hdf',
                         clusters_globstring='working/MC_gen_clusters_*.hdf', 
                         #relations_globstring='working/MC_gen_relations_*.hdf',
                         transition_clusters_globstring='working/MC_gen_transitions_*.hdf',
                         transitions_filename='working/MC_gen_transitions_list.pickle',
                         #tree_filename='working/MC_gen_tree.json.gz'
                        )

In [ ]:
files = glob.glob('../../../mnt/nfs-marketdepth/houghton/data/twitter/raw/posts_sample_20150618*')

# pick up from saved

In [ ]:
collection = homer.Homer(tree_filename='working/MC_gen_tree.json.gz')

In [ ]:
tree = collection.tree

In [ ]:
homer.tree.draw_series(node, 3)

In [ ]:
homer.tree.draw_series(tree.find("12"),3)

In [ ]:
collection.clusters.reset_index(drop=True).compute()

In [ ]:
df = collection.clusters.compute()

In [ ]:
df

In [ ]:
a = collection.clusters[collection.clusters['k']==7].compute()
a

In [ ]:
child = root.find(str(a.index[0]))
child.k_children

In [ ]:
b = collection.clusters[collection.clusters['k']==6][collection.clusters['Date']==1].compute()
b

In [ ]:
b = collection.clusters[collection.clusters['k']==5][collection.clusters['Date']==1].compute()
b

In [ ]:
parent = root.find('360431357983146192')
parent.k_children

In [ ]:
for n in homer.tree.walk_k_ancestry(parent):
    print(n, n.k_children)

In [ ]:
list(zip((1,2), (11, 22), (111, 222)))

In [ ]:
fig = plt.figure(figsize=(7,20))
ax = plt.gca()
ax_test = ax.twinx()
parent.layout(ax_test)
parent.set_bottom(100)
parent.set_center(100)
parent.draw(ax)

In [ ]:
parent.draw()

In [ ]:
a.k_children[2] is b.k_children[5]

In [ ]:
collection.clusters[collection.clusters['k']==7].compute()

In [ ]:
collection.clusters[(collection.clusters['k']==6)].compute()